In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
% cd /content

/content


In [ ]:
! git clone https://github.com/intel-isl/MiDaS.git

Cloning into 'MiDaS'...
remote: Enumerating objects: 250, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 394 (delta 58), reused 201 (delta 37), pack-reused 144
Receiving objects: 100% (394/394), 231.02 KiB | 4.53 MiB/s, done.
Resolving deltas: 100% (139/139), done.


In [ ]:
cd /content/MiDaS/

/content/MiDaS


In [ ]:
! git checkout -b old-state b00bf61f846d73fadc1f287293648db9f88d3615

Switched to a new branch 'old-state'


In [ ]:
# ! cp /content/drive/My\ Drive/planarcnn/8_image_0.png /content/MiDaS/input/
# ! cp /content/drive/My\ Drive/planarcnn/6_image_0.png /content/MiDaS/input/
# ! cp /content /drive/My\ Drive/output_source/image-001.jpg /content/MiDaS/input/

In [ ]:
! cp /content/drive/My\ Drive/pre_trained/model-f46da743.pt /content/MiDaS/

In [ ]:
% cd /content/MiDaS/

/content/MiDaS


In [ ]:
! mkdir ConstructionPPE/

In [ ]:
# ! cp -r /content/drive/My\ Drive/Depth\ Images\ using\ MiDaS /content/MiDaS/ConstructionPPE

In [ ]:
# ! cp -r /content/drive/My\ Drive/Annotated\ Images/ /content/MiDaS/ConstructionPPE/

In [ ]:
! cp  /content/drive/My\ Drive/ConstructionPPE/input_file_dir.txt /content/MiDaS/ConstructionPPE/

In [ ]:
! cp  /content/drive/My\ Drive/ConstructionPPE/MiDaS_file_dir.txt /content/MiDaS/ConstructionPPE/

In [ ]:
 ! python run.py

initialize
device: cuda
Loading weights:  model-f46da743.pt
Downloading: "https://github.com/facebookresearch/WSL-Images/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://download.pytorch.org/models/ig_resnext101_32x8-c38310e5.pth" to /root/.cache/torch/hub/checkpoints/ig_resnext101_32x8-c38310e5.pth
100% 340M/340M [00:09<00:00, 35.9MB/s]
start processing
finished


In [ ]:
% cd /content/MiDaS/

/content/MiDaS


In [ ]:
import os
import glob
import torch
import utils
import cv2

from torchvision.transforms import Compose
import torchvision.transforms as transforms
from midas.midas_net import MidasNet
from midas.transforms import Resize, NormalizeImage, PrepareForNet


In [ ]:
from torchsummary import summary

In [ ]:
input_path = "/content/drive/My Drive/ConstructionPPE/input_images"
output_path = "output"
# MODEL_PATH = "model.pt"
model_path = "model-f46da743.pt"

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = MidasNet(model_path, non_negative=True).to(device)


Loading weights:  model-f46da743.pt


Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master


In [ ]:
# print(summary(model.pretrained.layer1, (3, 416, 416)))

In [ ]:
# print(summary(model.pretrained.layer2, (256, 104, 104)))

In [ ]:
# print(summary(model.pretrained.layer3, (512, 52, 52)))

In [ ]:
# print(summary(model.pretrained.layer4, (1024, 26, 26)))

In [ ]:
# print(model)

In [ ]:
from torchsummary import summary
print(summary(model, (3, 416, 416)))

In [ ]:
print("initialize")

# select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: %s" % device)

# load network
model = MidasNet(model_path, non_negative=True)

transform = Compose(
    [
        
        Resize(
            608,
            608,
            resize_target=None,
            keep_aspect_ratio=False,
            ensure_multiple_of=32,
            resize_method="upper_bound",
            image_interpolation_method=cv2.INTER_CUBIC,
        ),
        # transforms.ToTensor(),
        NormalizeImage(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        PrepareForNet(),
    ]
)

model.to(device)

img_names = glob.glob(os.path.join(input_path, "*"))
num_images = len(img_names)


initialize
device: cuda
Loading weights:  model-f46da743.pt


Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master


In [ ]:
# from torchsummary import summary
# model.to(device)
# # sample_image = img_names[0]
# img = utils.read_image(sample_image)
# img_input = transform({"image": img})["image"]
# print(img_input.shape)
# print(summary(model, input_size = img_input.shape))

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np

In [ ]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from math import exp

def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()

def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = Variable(_2D_window.expand(channel, 1, window_size, window_size).contiguous())
    return window

def _ssim(img1, img2, window, window_size, channel, size_average = True):
    mu1 = F.conv2d(img1, window, padding = window_size//2, groups = channel)
    mu2 = F.conv2d(img2, window, padding = window_size//2, groups = channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1*mu2

    sigma1_sq = F.conv2d(img1*img1, window, padding = window_size//2, groups = channel) - mu1_sq
    sigma2_sq = F.conv2d(img2*img2, window, padding = window_size//2, groups = channel) - mu2_sq
    sigma12 = F.conv2d(img1*img2, window, padding = window_size//2, groups = channel) - mu1_mu2

    C1 = 0.01**2
    C2 = 0.03**2

    ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)

class SSIM(torch.nn.Module):
    def __init__(self, window_size = 11, size_average = True):
        super(SSIM, self).__init__()
        self.window_size = window_size
        self.size_average = size_average
        self.channel = 1
        self.window = create_window(window_size, self.channel)

    def forward(self, img1, img2):
        (_, channel, _, _) = img1.size()

        if channel == self.channel and self.window.data.type() == img1.data.type():
            window = self.window
        else:
            window = create_window(self.window_size, channel)
            
            if img1.is_cuda:
                window = window.cuda(img1.get_device())
            window = window.type_as(img1)
            
            self.window = window
            self.channel = channel


        return _ssim(img1, img2, window, self.window_size, channel, self.size_average)


In [ ]:


class ConstructionPPE(Dataset):

    def __init__(self, path):
        self.path = path

        super(ConstructionPPE, self).__init__()
        self.data, self.target = [], []
        self._load_data(200)
        

    def _load_data(self, lim):
        i = 0
        with open(os.path.join(self.path, 'input_file_dir.txt')) as f:
            for line in f.readlines():
                if i > lim:
                    break
                i += 1
                line = line.strip()
                imgs = os.path.join(self.path, 'Annotated Images/' + line)
                # print(imgs)
                image = Image.open(imgs)
                img_input = image.resize((384, 384), 2)
                trans = transforms.ToTensor()
                img_input = trans(np.array(img_input))
                self.data.append(img_input)
        
        i = 0
        with open(os.path.join(self.path, 'MiDaS_file_dir.txt')) as f:
            for line in f.readlines():
                if i > lim:
                    break
                i += 1
                line = line.strip()
                imgs = os.path.join(self.path, line)
                # print(imgs)
                image = Image.open(imgs)
                # transforms = Compose(
                #     [
                #         transforms.ResizeToTensor(),
                #         Resize(
                #             384,
                #             384,
                #             resize_target=None,
                #             keep_aspect_ratio=False,
                #             ensure_multiple_of=32,
                #             resize_method="upper_bound",
                #             image_interpolation_method=cv2.INTER_CUBIC,
                #         )
                #     ]
                # )
                # img_input = transform({"image": image})["image"]
                img_input = image.resize((384, 384), 2)
                # print(img_input.size)
                trans = transforms.ToTensor()
                img_input = trans(np.array(img_input))
                img_input = np.array(img_input).astype(np.float32) / 65536
                # print(len(img_input.split()))
                self.target.append(img_input)
        
    def get_data(self):
        return list(zip(self.data, self.target))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_name = self.data[idx]
        label_name = self.target[idx]
        image = Image.open(image_name)
        label = Image.open(label_name)
        return (image, label)


In [ ]:

def data_loader(data, shuffle=True, batch_size=1, num_workers=1, cuda=False):

    loader_args = {
        'shuffle': shuffle,
        'batch_size': batch_size
    }

    # If GPU exists
    if cuda:
        loader_args['num_workers'] = num_workers
        loader_args['pin_memory'] = True
    
    return DataLoader(data, **loader_args)

In [ ]:
dataclass = ConstructionPPE("/content/drive/My Drive/ConstructionPPE/")
dataset = dataclass.get_data()

In [ ]:
# from torchvision import datasets

In [ ]:
# dataset = datasets.ImageFolder(root='ConstructionPPE/', transform=transform)

In [ ]:
loader = data_loader(dataset, shuffle=True, batch_size = 4, num_workers=16, cuda = False)

In [ ]:
sample = next(iter(loader))
print(len(sample))

2


In [ ]:
print(sample[0])
print('label now..')
print(sample[1])

In [ ]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
ssim = SSIM()
criterion = ssim.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0)

In [ ]:
from tqdm import tqdm

In [ ]:
def train(model, loader, device, optimizer, criterion, l1_factor=0.0):
    model.train()
    pbar = tqdm(loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar, 0):
        # Get samples
        data, target = data.to(device), target.to(device)

        # Set gradients to zero before starting backpropagation
        optimizer.zero_grad()

        # Predict output
        y_pred = model(data)
        # target = target.squeeze()
        # y_pred = y_pred.squeeze()
        # Calculate loss
        # print(y_pred.shape, target.shape)
        # print(y_pred)
        # print(target)
        target = target.float()
        y_pred = torch.unsqueeze(y_pred, 1)
        loss = criterion(y_pred, target)    
        # loss += l1_regularization(model, l1_factor)

        # Perform backpropagation
        loss.backward()
        optimizer.step()

        # Update Progress Bar
        pred = y_pred.argmax(dim=1, keepdim=True)
        # correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(
            desc=f'Loss={loss.item():0.2f} Batch_ID={batch_idx}'
        )
    print('total loss:', loss)
    return loss

In [ ]:
for i in range(0, 4):
    train(model, loader, device, optimizer, criterion, l1_factor = 0)

  0%|          | 0/51 [00:00<?, ?it/s]

total loss: tensor(5.9835e-09, device='cuda:0', grad_fn=<MeanBackward0>)


  0%|          | 0/51 [00:00<?, ?it/s]

total loss: tensor(4.4403e-08, device='cuda:0', grad_fn=<MeanBackward0>)


  0%|          | 0/51 [00:00<?, ?it/s]

total loss: tensor(8.8404e-08, device='cuda:0', grad_fn=<MeanBackward0>)


Loss=0.00 Batch_ID=50: 100%|██████████| 51/51 [00:51<00:00,  1.01s/it]

total loss: tensor(3.9374e-07, device='cuda:0', grad_fn=<MeanBackward0>)
